In [32]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
%autoreload

In [34]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [35]:
import pandas as pd

from flora_tools.radio_configuration import RadioConfiguration
from flora_tools.gloria_math import GloriaMath

payloads = [8, 16, 32, 64, 128, 255]

## Rx Variable Time

In [36]:
columns = ['modulation_name']
columns.extend([str(i) for i in payloads])

rx_df = pd.DataFrame(columns=columns)
rx_df.index.name = 'modulation'

for i in range(0, 10):
    item = [RadioConfiguration(i).modulation_name]
    for j in payloads:
        item.append(GloriaMath().calculate_total_rx_time(i, j)) 
    rx_df.loc[i] = item
    
rx_df.style

,modulation_name,8,16,32,64,128,255
modulation,,,,,,,
0,SF12,1.1331,1.46078,1.9523,2.93534,5.06526,9.16126
1,SF11,0.570789,0.734629,1.06231,1.63575,2.78263,5.07639
2,SF10,0.282347,0.364267,0.487147,0.732907,1.26539,2.33035
3,SF9,0.141637,0.182597,0.264517,0.407877,0.694597,1.26804
4,SF8,0.0818579,0.102338,0.143298,0.225218,0.389058,0.716738
5,SF7,0.041372,0.056732,0.077212,0.123292,0.220572,0.404892
6,SF6,0.0237999,0.0314799,0.0442799,0.0724399,0.1262,0.23372
7,SF5,0.0134114,0.0172514,0.0249314,0.0415714,0.0748514,0.140131
8,FSK 125k,0.00142448,0.00193648,0.00296048,0.00500848,0.00910448,0.0172325


In [37]:
rx_offset_df = pd.DataFrame(columns=['modulation_name', 'offset'])
rx_offset_df.index.name = 'modulation'

for i in range(0, 10):
    item = [RadioConfiguration(i).modulation_name]
    item.append(GloriaMath().calculate_rx_offset(i)) 
    rx_offset_df.loc[i] = item
    
    
rx_offset_df.style

,modulation_name,offset
modulation,,
0,SF12,0.0475136
1,SF11,0.0237568
2,SF10,0.0118784
3,SF9,0.0059392
4,SF8,0.0029696
5,SF7,0.0014848
6,SF6,0.0009472
7,SF5,0.0004736
8,FSK 125k,2.56e-05


## Tx Variable Time

In [38]:
tx_df = pd.DataFrame(columns=columns)
tx_df.index.name = 'modulation'

for i in range(0, 10):
    item = [RadioConfiguration(i).modulation_name]
    for j in payloads:
        item.append(GloriaMath().calculate_total_tx_time(i, j)) 
    tx_df.loc[i] = item
    
tx_df.style

,modulation_name,8,16,32,64,128,255
modulation,,,,,,,
0,SF12,0.931924,1.2596,1.75112,2.73416,4.86408,8.96008
1,SF11,0.444482,0.608322,0.936002,1.50944,2.65632,4.95008
2,SF10,0.227791,0.309711,0.432591,0.678351,1.21083,2.27579
3,SF9,0.115924,0.156884,0.238804,0.382164,0.668884,1.24232
4,SF8,0.0688479,0.0893279,0.130288,0.212208,0.376048,0.703728
5,SF7,0.033063,0.048423,0.068903,0.114983,0.212263,0.396583
6,SF6,0.0194108,0.0270908,0.0398908,0.0680508,0.121811,0.229331
7,SF5,0.0115591,0.0153991,0.0230791,0.0397191,0.0729991,0.138279
8,FSK 125k,0.00133613,0.00184813,0.00287213,0.00492013,0.00901613,0.0171441


## Compare Rx$\rightarrow$Tx vs Tx$\rightarrow$Rx Timings

In [39]:
rx2tx_time = pd.DataFrame(columns=['8', '16', '32', '64', '128', '255'])
rx2tx_time.index.name = 'modulation'

for j in payloads:
    rx2tx_time.loc[:, str(j)] = rx_df.loc[:, str(j)] - rx_offset_df.offset + GloriaMath().rx_irq_time + GloriaMath().tx_setup_time
    
rx2tx_time.style

,8,16,32,64,128,255
modulation,,,,,,
0,1.08667,1.41435,1.90587,2.88891,5.01883,9.11483
1,0.548111,0.711951,1.03963,1.61307,2.75995,5.05371
2,0.271547,0.353467,0.476347,0.722107,1.25459,2.31955
3,0.136777,0.177737,0.259657,0.403017,0.689737,1.26318
4,0.0799675,0.100448,0.141408,0.223328,0.387168,0.714848
5,0.0409664,0.0563264,0.0768064,0.122886,0.220166,0.404486
6,0.0239319,0.0316119,0.0444119,0.0725719,0.126332,0.233852
7,0.014017,0.017857,0.025537,0.042177,0.075457,0.140737
8,0.00247811,0.00299011,0.00401411,0.00606211,0.0101581,0.0182861


In [40]:
tx2rx_time = pd.DataFrame(columns=['8', '16', '32', '64', '128', '255'])
tx2rx_time.index.name = 'modulation'

for j in payloads:
    tx2rx_time.loc[:, str(j)] = tx_df.loc[:, str(j)] + GloriaMath().tx_irq_time + GloriaMath().rx_setup_time + rx_offset_df.offset
    
tx2rx_time.style

,8,16,32,64,128,255
modulation,,,,,,
0,0.980162,1.30784,1.79936,2.7824,4.91232,9.00832
1,0.468964,0.632804,0.960484,1.53392,2.6808,4.97456
2,0.240395,0.322315,0.445195,0.690955,1.22343,2.28839
3,0.122588,0.163548,0.245468,0.388828,0.675548,1.24899
4,0.0725423,0.0930223,0.133982,0.215902,0.379742,0.707422
5,0.0352726,0.0506326,0.0711126,0.117193,0.214473,0.398793
6,0.0210828,0.0287628,0.0415628,0.0697228,0.123483,0.231003
7,0.0127575,0.0165975,0.0242775,0.0409175,0.0741975,0.139477
8,0.00208649,0.00259849,0.00362249,0.00567049,0.00976649,0.0178945


In [41]:
min_slot_time = pd.concat([rx2tx_time, tx2rx_time]).max(level=0)
min_slot_time.style

,8,16,32,64,128,255
modulation,,,,,,
0,1.08667,1.41435,1.90587,2.88891,5.01883,9.11483
1,0.548111,0.711951,1.03963,1.61307,2.75995,5.05371
2,0.271547,0.353467,0.476347,0.722107,1.25459,2.31955
3,0.136777,0.177737,0.259657,0.403017,0.689737,1.26318
4,0.0799675,0.100448,0.141408,0.223328,0.387168,0.714848
5,0.0409664,0.0563264,0.0768064,0.122886,0.220166,0.404486
6,0.0239319,0.0316119,0.0444119,0.0725719,0.126332,0.233852
7,0.014017,0.017857,0.025537,0.042177,0.075457,0.140737
8,0.00247811,0.00299011,0.00401411,0.00606211,0.0101581,0.0182861


In [42]:
to_timer_ticks = lambda x: int(x * 8E6)

min_slot_time.applymap(to_timer_ticks)

,8,16,32,64,128,255
modulation,,,,,,
0,8693360,11314800,15246960,23111280,40150640,72918640
1,4384887,5695607,8317047,12904567,22079607,40429687
2,2172378,2827738,3810778,5776858,10036698,18556378
3,1094219,1421899,2077259,3224139,5517899,10105419
4,639740,803580,1131260,1786620,3097340,5718780
5,327731,450611,614451,983091,1761331,3235891
6,191455,252895,355295,580575,1010655,1870815
7,112136,142856,204296,337416,603656,1125896
8,19824,23920,32112,48496,81264,146288


## Gloria Slot Timings

In [43]:
from lwb_math import repetitions, max_hops

In [44]:
lwb_slot_time = pd.DataFrame(columns=['8', '16', '32', '64', '128', '255'])
lwb_slot_time.index.name = 'modulation'

for i in range(0,10):
    item = []
    for j in payloads:
        item.append(GloriaMath().calculate_flood(i, j, repetitions[i], max_hops[i])['time'])
    lwb_slot_time.loc[i] = item

lwb_slot_time.style


,8,16,32,64,128,255
modulation,,,,,,
0,3.31009,4.29313,5.76769,8.71681,15.1066,27.3946
1,1.67066,2.16218,3.14522,4.86554,8.30618,15.1875
2,0.82909,1.07485,1.44349,2.18077,3.77821,6.97309
3,0.829323,1.07508,1.5666,2.42676,4.14708,7.58772
4,0.485494,0.608374,0.854134,1.34565,2.32869,4.29477
5,0.250003,0.342163,0.465043,0.741523,1.3252,2.43112
6,0.147258,0.193338,0.270138,0.439098,0.761658,1.40678
7,0.0872951,0.110335,0.156415,0.256255,0.455935,0.847615
8,0.0176137,0.0206857,0.0268297,0.0391177,0.0636937,0.112462


In [45]:
to_timer_ticks = lambda x: int(x * 8E6)

lwb_slot_time.applymap(to_timer_ticks)

,8,16,32,64,128,255
modulation,,,,,,
0,26480744,34345064,46141544,69734504,120852584,219156584
1,13365272,17297432,25161752,38924312,66449432,121499672
2,6632718,8598798,11547918,17446158,30225678,55784718
3,6634587,8600667,12532827,19414107,33176667,60701787
4,3883952,4866992,6833072,10765232,18629552,34358192
5,2000021,2737301,3720341,5932181,10601621,19448981
6,1178066,1546706,2161106,3512786,6093266,11254226
7,698361,882681,1251321,2050041,3647481,6780921
8,140909,165485,214637,312941,509549,899693


### Energy

In [46]:
lwb_slot_energy = pd.DataFrame(columns=['8', '16', '32', '64', '128', '255'])
lwb_slot_energy.index.name = 'modulation'

for i in range(0,10):
    item = []
    for j in payloads:
        item.append(GloriaMath().calculate_flood_energy(i, j, repetitions[i], max_hops[i]))
    lwb_slot_energy.loc[i] = item

lwb_slot_energy.style

,8,16,32,64,128,255
modulation,,,,,,
0,0.141668,0.186117,0.252792,0.38614,0.675061,1.23068
1,0.070142,0.0923667,0.136816,0.214603,0.370176,0.681322
2,0.0351547,0.0462671,0.0629356,0.0962727,0.168503,0.312964
3,0.0341076,0.0448483,0.0663296,0.103922,0.179106,0.329476
4,0.0200201,0.0253905,0.0361311,0.0576124,0.100575,0.1865
5,0.0100282,0.0140559,0.0194263,0.0315095,0.0570185,0.105351
6,0.00587444,0.00788831,0.0112448,0.018629,0.0327261,0.0609203
7,0.00345612,0.00446306,0.00647693,0.0108403,0.0195671,0.036685
8,0.000536791,0.000671049,0.000939565,0.0014766,0.00255066,0.00468201


In [47]:
lwb_slot_energy_per_byte = lwb_slot_energy
energy = 12 * 3600 * 3.7

for i in range(0,10):
    for j in payloads:
        lwb_slot_energy_per_byte.loc[i,str(j)] /= j

energy / lwb_slot_energy_per_byte

,8,16,32,64,128,255
modulation,,,,,,
0,9.026180e+06,1.374101e+07,2.023359e+07,2.649236e+07,3.030764e+07,3.311926e+07
1,1.823045e+07,2.768789e+07,3.738505e+07,4.766835e+07,5.526973e+07,5.982368e+07
2,3.637408e+07,5.527560e+07,8.127162e+07,1.062581e+08,1.214193e+08,1.302361e+08
3,3.749077e+07,5.702430e+07,7.711313e+07,9.843705e+07,1.142311e+08,1.237093e+08
4,6.387169e+07,1.007244e+08,1.415644e+08,1.775617e+08,2.034254e+08,2.185477e+08
5,1.275126e+08,1.819474e+08,2.632972e+08,3.246564e+08,3.588222e+08,3.868877e+08
6,2.176752e+08,3.242062e+08,4.548676e+08,5.491319e+08,6.251749e+08,6.690579e+08
7,3.699868e+08,5.730240e+08,7.897072e+08,9.436767e+08,1.045608e+09,1.111058e+09
8,2.382158e+09,3.811109e+09,5.443880e+09,6.927925e+09,8.021255e+09,8.705488e+09


~ 3 TB of data (FSK 200 kBit/s, 255 Bytes, 12000 mAh @ 3.7 Volts, 10mW TX)

32 Byte datapoint:

In [48]:
energy / lwb_slot_energy_per_byte / 100 / 32

,8,16,32,64,128,255
modulation,,,,,,
0,2820.681319,4.294065e+03,6.322996e+03,8.278864e+03,9.471137e+03,1.034977e+04
1,5697.014402,8.652466e+03,1.168283e+04,1.489636e+04,1.727179e+04,1.869490e+04
2,11366.899487,1.727362e+04,2.539738e+04,3.320567e+04,3.794352e+04,4.069879e+04
3,11715.865038,1.782009e+04,2.409785e+04,3.076158e+04,3.569721e+04,3.865916e+04
4,19959.903311,3.147638e+04,4.423887e+04,5.548803e+04,6.357044e+04,6.829615e+04
5,39847.690743,5.685856e+04,8.228039e+04,1.014551e+05,1.121319e+05,1.209024e+05
6,68023.511991,1.013145e+05,1.421461e+05,1.716037e+05,1.953672e+05,2.090806e+05
7,115620.878408,1.790700e+05,2.467835e+05,2.948990e+05,3.267525e+05,3.472057e+05
8,744424.280056,1.190971e+06,1.701212e+06,2.164977e+06,2.506642e+06,2.720465e+06


### Speed

In [49]:
lwb_slot_bitrate = pd.DataFrame(columns=['8', '16', '32', '64', '128', '255'])
lwb_slot_bitrate.index.name = 'modulation'

for i in range(0,10):
    item = []
    for j in payloads:
        item.append(GloriaMath().calculate_flood_bitrate(i, j, repetitions[i], max_hops[i]))
    lwb_slot_bitrate.loc[i] = item

lwb_slot_bitrate.style

,8,16,32,64,128,255
modulation,,,,,,
0,19.3348,29.8151,44.3852,58.7371,67.7851,74.4673
1,38.3082,59.1995,81.3934,105.23,123.282,134.321
2,77.1931,119.086,177.348,234.779,271.028,292.553
3,77.1713,119.061,163.411,210.981,246.921,268.855
4,131.824,210.397,299.719,380.484,439.731,474.996
5,255.997,374.091,550.487,690.471,772.712,839.118
6,434.611,662.052,947.663,1166.03,1344.43,1450.12
7,733.145,1160.1,1636.67,1998.01,2245.93,2406.75
8,3633.54,6187.86,9541.67,13088.7,16076.9,18139.5


## Gloria Slot + Gloria ACK Timings

In [50]:
lwb_ack_slot_time = pd.DataFrame(columns=['8', '16', '32', '64', '128', '255'])
lwb_ack_slot_time.index.name = 'modulation'

for i in range(0,10):
    item = []
    for j in payloads:
        item.append(GloriaMath().calculate_flood(i, j, repetitions[i], max_hops[i], ack=True)['time'])
    lwb_ack_slot_time.loc[i] = item

lwb_ack_slot_time.style


,8,16,32,64,128,255
modulation,,,,,,
0,6.36345,7.34649,8.82105,11.7702,18.1599,30.4479
1,3.21156,3.70308,4.68612,6.40644,9.84708,16.7284
2,1.59191,1.83767,2.20631,2.94359,4.54103,7.73591
3,1.59795,1.84371,2.33523,3.19539,4.91571,8.35635
4,0.877625,1.00051,1.24627,1.73779,2.72083,4.68691
5,0.48247,0.57463,0.69751,0.97399,1.55767,2.66359
6,0.271067,0.317147,0.393947,0.562907,0.885467,1.53059
7,0.153612,0.176652,0.222732,0.322572,0.522252,0.913932
8,0.0300566,0.0331286,0.0392726,0.0515606,0.0761366,0.124905


In [51]:
to_timer_ticks = lambda x: int(x * 8E6)

lwb_ack_slot_time.applymap(to_timer_ticks)

,8,16,32,64,128,255
modulation,,,,,,
0,50907602,58771922,70568402,94161362,145279442,243583442
1,25692466,29624626,37488946,51251506,78776626,133826866
2,12735262,14701342,17650462,23548702,36328222,61887262
3,12783598,14749678,18681838,25563118,39325678,66850798
4,7021002,8004042,9970122,13902282,21766602,37495242
5,3859758,4597038,5580078,7791918,12461358,21308718
6,2168539,2537179,3151579,4503259,7083739,12244699
7,1228892,1413212,1781852,2580572,4178012,7311452
8,240452,265028,314180,412484,609092,999236
